<a href="https://colab.research.google.com/github/zeroam/jupyter-notebook/blob/master/dive_into_deep_learning/3.10.mlp_gluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mxnet==1.6.0b20190915
!pip install d2l

In [0]:
import d2l
from mxnet import gluon, init, autograd
from mxnet.gluon import loss as gloss, nn

In [0]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

In [0]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        y = y.astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum().asscalar()
        n += y.size
    return acc_sum / n

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            if trainer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                # This will be illustrated in the next section
                trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [7]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})
num_epochs = 10
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
          None, trainer)

epoch 1, loss 0.7985, train acc 0.704, test acc 0.818
epoch 2, loss 0.4845, train acc 0.822, test acc 0.850
epoch 3, loss 0.4284, train acc 0.843, test acc 0.861
epoch 4, loss 0.3962, train acc 0.854, test acc 0.857
epoch 5, loss 0.3695, train acc 0.865, test acc 0.870
epoch 6, loss 0.3528, train acc 0.869, test acc 0.876
epoch 7, loss 0.3366, train acc 0.875, test acc 0.877
epoch 8, loss 0.3234, train acc 0.880, test acc 0.881
epoch 9, loss 0.3132, train acc 0.884, test acc 0.883
epoch 10, loss 0.3066, train acc 0.886, test acc 0.883
